In [2]:
library(data.table)
library(caTools) # For train-test split
library(nnet) # For multinominal models

In [3]:
setwd("Downloads")

# Continuous X
## Predict pass/fail based on hours of study

In [4]:
# Read the dataset
passexam <- fread("passexam.csv")

In [5]:
# Factor "Outcome" and change (0,1) to ("F","P")
passexam$Outcome <- factor(passexam$Outcome, labels=c("F","P"))
summary(passexam)

     Hours       Outcome
 Min.   :0.500   F:10   
 1st Qu.:1.688   P:10   
 Median :2.625          
 Mean   :2.788          
 3rd Qu.:4.062          
 Max.   :5.500          

In [6]:
# Fit logistic regression model
passexam.glm <- glm(Outcome ~ Hours, data=passexam, family=binomial)
summary(passexam.glm)


Call:
glm(formula = Outcome ~ Hours, family = binomial, data = passexam)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)  
(Intercept)  -4.0777     1.7610  -2.316   0.0206 *
Hours         1.5046     0.6287   2.393   0.0167 *
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 27.726  on 19  degrees of freedom
Residual deviance: 16.060  on 18  degrees of freedom
AIC: 20.06

Number of Fisher Scoring iterations: 5


- Hours is statistically significant
- z = -4.08 + 1.50(Hours)
- Pr(Y=Pass) = 1 / (1+e^-z)

In [7]:
# Odds-Ratio and Odds-Ratio Confidence Interval
OR <- exp(coef(passexam.glm))
OR
OR.CI <- exp(confint(passexam.glm))
OR.CI

(Intercept)       Hours 
 0.01694617  4.50255687

Waiting for profiling to be done...



,2.5 %,97.5 %
(Intercept),0.0001868263,0.2812875
Hours,1.6978380343,23.2228735


- Since the Odds-Ratio confidence interval of "Hours" does not include 1, the odds-ratio is satistically significant
- Odds-Ratio of "Hours" is 4.5, meaning that 1 more hour of study will increase the odds of passing by 4.5

In [8]:
# Confusion Matrix
prob <- predict(passexam.glm, type='response')
threshold <- sum(passexam$Outcome=="P")/length(passexam$Outcome)
Y.hat <- ifelse(prob > threshold, "P", "F")
table(passexam$Outcome, Y.hat)
# Overall Accuracy
mean(Y.hat==passexam$Outcome)

   Y.hat
    F P
  F 8 2
  P 2 8

[1] 0.8

- Model's accuracy is 80%

# Continuous & Categorical X 
## Predict Loan default based on Gender, Avgbal, Avgoutstandingbal & Income

In [9]:
# Read the dataset and set strings as factors
default <- fread("default.csv", stringsAsFactors = TRUE)
summary(default)

 Default    Gender       AvgBal           Income     
 No :9667   F:2944   Min.   :   0.0   Min.   :  772  
 Yes: 333   M:7056   1st Qu.: 482.0   1st Qu.:21341  
                     Median : 824.0   Median :34553  
                     Mean   : 835.4   Mean   :33517  
                     3rd Qu.:1166.2   3rd Qu.:43808  
                     Max.   :2654.0   Max.   :73554  

In [10]:
# Check levels of categorical variables and set "M" as the baseline
levels(default$Default)
levels(default$Gender)
default$Gender <- relevel(default$Gender, ref="M")
levels(default$Gender)

[1] "No"  "Yes"

[1] "F" "M"

[1] "M" "F"

In [11]:
# Create a 70-30 train-test split for "Default"
set.seed(2014)
train <- sample.split(Y=default$Default, SplitRatio=0.7)
train_set <- subset(default, train==TRUE)
test_set <- subset(default, train==FALSE)

In [12]:
# Verify that the proportion is similar in both train and test sets
prop.table(table(train_set$Default))
prop.table(table(test_set$Default))


        No        Yes 
0.96671429 0.03328571 


        No        Yes 
0.96666667 0.03333333 

In [13]:
# Fit the logistic regression model
default.glm <- glm(Default~., family=binomial, data=train_set)
summary(default.glm)


Call:
glm(formula = Default ~ ., family = binomial, data = train_set)

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.103e+01  6.000e-01 -18.388   <2e-16 ***
GenderF     -5.572e-01  2.786e-01  -2.000   0.0455 *  
AvgBal       5.777e-03  2.802e-04  20.615   <2e-16 ***
Income       4.654e-06  9.774e-06   0.476   0.6340    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 2043.8  on 6999  degrees of freedom
Residual deviance: 1108.6  on 6996  degrees of freedom
AIC: 1116.6

Number of Fisher Scoring iterations: 8


- AvgBal & Gender are statistically significant but not Income
- Refit the model after removing the non-statistically significant variable (Income) 

In [14]:
# Refit the logistic regression model after removing "Income"
default.glm2 <- glm(Default~.-Income, family=binomial, data=train_set)
summary(default.glm2)


Call:
glm(formula = Default ~ . - Income, family = binomial, data = train_set)

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.084e+01  4.479e-01 -24.210  < 2e-16 ***
GenderF     -6.606e-01  1.735e-01  -3.806 0.000141 ***
AvgBal       5.776e-03  2.801e-04  20.626  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 2043.8  on 6999  degrees of freedom
Residual deviance: 1108.8  on 6997  degrees of freedom
AIC: 1114.8

Number of Fisher Scoring iterations: 8


- Now all variables are satistically significant
- z = -10.84 - 0.66(Gender) - 0.0058(AvgBal)
- Pr(Y=Default) = 1 / (1+e^-z)

In [15]:
# Odds-Ratio and Odds-Ratio Confidence Interval
OR <- exp(coef(default.glm2))
OR
OR.CI <- exp(confint(default.glm2))
OR.CI
## - Since the Odds-Ratio confidence interval of both variables do not include 1, the odds-ratio is satistically significant
## - Since the Odds-Ratio for Gender=F is 0.517, the odds of default for females are 100% - 51.7% = 48.3% less compared to men
## - Every unit increase in "AvgBal" will lead to 1.0058 increase in odds of default

(Intercept)      GenderF       AvgBal 
1.951025e-05 5.165542e-01 1.005793e+00

Waiting for profiling to be done...



,2.5 %,97.5 %
(Intercept),7.819165e-06,4.534699e-05
GenderF,3.657962e-01,7.227697e-01
AvgBal,1.005260e+00,1.006365e+00


In [16]:
# Confusion Matrix for Train set
threshold <- 0.5
prob.train <- predict(default.glm2, type="response", newdata=train_set)
predict.train <- ifelse(prob.train>threshold, "Yes", "No")
table.train <- table(train_set$Default, predict.train)
table.train
round(prop.table(table.train), 3)
cat("Train set overall accuracy:", round(mean(predict.train == train_set$Default),3))

     predict.train
        No  Yes
  No  6738   29
  Yes  164   69

     predict.train
         No   Yes
  No  0.963 0.004
  Yes 0.023 0.010

Train set overall accuracy: 0.972

In [17]:
# Confusion Matrix for Train set
prob.test <- predict(default.glm2, type="response", newdata=test_set)
predict.test <- ifelse(prob.test>threshold, "Yes", "No")
table.test <- table(test_set$Default, predict.test)
table.test
round(prop.table(table.test),3)
cat("Test set overall accuracy:", round(mean(predict.test == test_set$Default),3))

     predict.test
        No  Yes
  No  2890   10
  Yes   64   36

     predict.test
         No   Yes
  No  0.963 0.003
  Yes 0.021 0.012

Test set overall accuracy: 0.975

# Multi-Categorical Y
## Predict Ratings (Bad,Neutral,Good)

In [18]:
# Read the dataset and convert strings to categorical
rating.dt <- fread("rating.csv", stringsAsFactors=TRUE)

In [19]:
# Check levels of rating and change baseline to "Neutral"
levels(rating.dt$Rating)
rating.dt$Rating <- relevel(rating.dt$Rating, ref="Neutral")
levels(rating.dt$Rating)

[1] "Bad"     "Good"    "Neutral"

[1] "Neutral" "Bad"     "Good"

In [20]:
# Fit the linear regression model
rating.glm <- multinom(Rating~.-Cust, data=rating.dt)
summary(rating.glm)

# weights:  18 (10 variable)
initial  value 195.552987 
iter  10 value 125.792824
final  value 123.862307 
converged


Call:
multinom(formula = Rating ~ . - Cust, data = rating.dt)

Coefficients:
     (Intercept)        WTQ       WTP LocationB   LocationC
Bad    -3.911087  0.3353235 0.0714015 0.1522743 -0.03359073
Good    1.195214 -0.2388393 0.1420355 0.1017307  0.39957803

Std. Errors:
     (Intercept)        WTQ       WTP LocationB LocationC
Bad    0.9031107 0.06112252 0.2287039 0.5611857 0.5701487
Good   0.6438784 0.05364305 0.2053201 0.5366026 0.5208776

Residual Deviance: 247.7246 
AIC: 267.7246 

In [21]:
# Manually calculate the p-values for the coefficients
z <- summary(rating.glm)$coefficients / summary(rating.glm)$standard.errors
p_value <- (1-pnorm(abs(z), 0, 1))*2
p_value

,(Intercept),WTQ,WTP,LocationB,LocationC
Bad,1.486471e-05,4.109339e-08,0.7548881,0.7861266,0.9530192
Good,6.341463e-02,8.492339e-06,0.4890781,0.8496359,0.4430074


- Only "WTQ" is statistically significant
- Refit the model by removing all statistically insignificant variables

In [22]:
# Refit the linear regression model without any statistically insignificant variables / with only "WTQ"
rating.glm2 <- multinom(Rating~WTQ, data=rating.dt)
summary(rating.glm2)

# weights:  9 (4 variable)
initial  value 195.552987 
iter  10 value 124.530286
final  value 124.525995 
converged


Call:
multinom(formula = Rating ~ WTQ, data = rating.dt)

Coefficients:
     (Intercept)        WTQ
Bad    -3.730275  0.3354517
Good    1.675657 -0.2427254

Std. Errors:
     (Intercept)        WTQ
Bad    0.7605860 0.06116371
Good   0.3931501 0.05348945

Residual Deviance: 249.052 
AIC: 257.052 

In [23]:
# Manually calculate the p-values for the coefficients
z2 <- summary(rating.glm2)$coefficients / summary(rating.glm2)$standard.errors
p_value2 <- (1-pnorm(abs(z2), 0, 1))*2
p_value2

,(Intercept),WTQ
Bad,9.367736e-07,4.146681e-08
Good,2.024869e-05,5.683932e-06


- Now all variables are statistically significant
- z_bad = -3.73 + 0.34(WTQ)
- Pr(Y=Bad) = 1 / (1+e^-z_bad)
- z_good = 1.68 - 0.24(WTQ)
- Pr(Y=Good) = 1 / (1+e^-z_good)
- Pr(Y=Neutral) 1 / (1+e^z_bad+z^z_good)

In [24]:
# Odds-Ratio and Odds-Ratio Confidence Interval
OR <- exp(coef(rating.glm2))
OR
OR.CI <- exp(confint(rating.glm2))
OR.CI

,(Intercept),WTQ
Bad,0.02398624,1.3985719
Good,5.34230338,0.7844869


, , Bad

                  2.5 %   97.5 %
(Intercept) 0.005401945 0.106506
WTQ         1.240572531 1.576694

, , Good

                2.5 %     97.5 %
(Intercept) 2.4721716 11.5445890
WTQ         0.7064077  0.8711963

- Since the Odds-Ratio confidence interval of both variables do not include 1, the odds-ratio is statistically significant
- Every unit increase in "WTQ" will lead to 1.399 increase in odds of Bad reviews
- Every unit increase in "WTQ" will lead to 0.784 increase in odds of Good reviews

In [25]:
## Model-predicted service rating for each of the case in the dataset
prob <- predict(rating.glm2, type="prob")
predicted.rating <- predict(rating.glm2)
predicted.rating

[1] Neutral Good    Good    Good    Good    Bad     Good    Bad     Good   
 [10] Neutral Good    Good    Good    Good    Good    Good    Good    Good   
 [19] Good    Good    Good    Good    Good    Good    Good    Good    Good   
 [28] Good    Good    Neutral Neutral Neutral Good    Good    Good    Good   
 [37] Good    Good    Neutral Neutral Good    Good    Good    Neutral Neutral
 [46] Neutral Good    Good    Good    Good    Neutral Good    Good    Good   
 [55] Neutral Neutral Good    Good    Bad     Good    Good    Bad     Good   
 [64] Bad     Bad     Neutral Good    Bad     Bad     Good    Neutral Good   
 [73] Neutral Neutral Neutral Good    Bad     Good    Bad     Neutral Bad    
 [82] Good    Neutral Good    Neutral Good    Good    Bad     Bad     Neutral
 [91] Bad     Bad     Bad     Neutral Good    Bad     Neutral Good    Good   
[100] Good    Good    Neutral Good    Neutral Good    Neutral Neutral Neutral
[109] Neutral Bad     Neutral Bad     Bad     Bad     Bad     Bad     Neutral
[118] Bad     Bad     Bad     Bad     Bad     Bad     Bad     Bad     Neutral
[127] Neutral Bad     Bad     Bad     Bad     Bad     Bad     Bad     Good   
[136] Neutral Bad     Neutral Good    Neutral Bad     Bad     Bad     Bad    
[145] Bad     Bad     Bad     Bad     Bad     Bad     Neutral Neutral Bad    
[154] Bad     Good    Bad     Bad     Bad     Bad     Bad     Bad     Bad    
[163] Bad     Bad     Bad     Bad     Bad     Bad     Bad     Bad     Bad    
[172] Bad     Bad     Bad     Bad     Bad     Bad     Bad    
Levels: Neutral Bad Good